In [1]:
import requests
import pyarrow as pa
import pyarrow.fs

In [2]:
#Q8:
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 19867217020 (18.50 GB)
DFS Remaining: 19338231808 (18.01 GB)
DFS Used: 528985212 (504.48 MB)
DFS Used%: 2.66%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.24.0.3:9866 (project-4-asian-cheese-1-dn-2.project-4-asian-cheese-1_default)
Hostname: 56bca2a3d1a2
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 264936291 (252.66 MB)
Non DFS Used: 15870223517 (14.78 GB)
DFS Remaining: 9669115904 (9.01 GB)
DFS Used%: 1.03%

In [3]:
#Q9:
r_b = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")  
b_size = r_b.json()["FileStatus"]["blockSize"] 
len= r_b.json()["FileStatus"]["length"] 
r_b.json()

b_Pt = 0
b_valid = []
ans = {}

while (b_Pt <= len): #looping for block pointer
    r_b = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&noredirect=true&offset=" + str(b_Pt))
    
    if r_b.status_code ==403:
        
        if "lost" not in ans:
            ans["lost"] =1
            b_Pt += b_size
        else:
            ans["lost"] += 1
            b_Pt += b_size
    else:
        id = r_b.json()["Location"][7:19]
        
        if id not in ans:
            ans[id] = 0
            
        ans[id] +=1
        b_Pt += b_size
        addy = r_b.json()['Location']
        b_valid.append(addy[addy.index("offset=") + 7:])

ans



{'56bca2a3d1a2': 84, '0b534b8c4501': 83}

In [4]:
#Q10:
b_size = 1048576
counter = 0

hdfs = pa.fs.HadoopFileSystem("boss", 9000)
file = hdfs.open_input_file("hdfs://boss:9000/single.csv")

for i in b_valid:
    
    stream = file.get_stream(int(i), b_size)
    b_str = stream.read().decode('utf-8')
    
    if "Single Family" in b_str:
        counter +=1
        
counter

2024-03-16 07:28:07,722 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


167